In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
!mkdir data
!wget -P data/ https://raw.githubusercontent.com/dewirizki/DSU/main/Dataset/catatan_lokasi.csv
!wget -P data/ https://raw.githubusercontent.com/dewirizki/DSU/main/Dataset/data_profil.csv

mkdir: cannot create directory ‘data’: File exists
--2022-04-02 15:22:36--  https://raw.githubusercontent.com/dewirizki/DSU/main/Dataset/catatan_lokasi.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73132 (71K) [text/plain]
Saving to: ‘data/catatan_lokasi.csv.2’

catatan_lokasi.csv. 100%[===================>]  71.42K  --.-KB/s    in 0.001s  

2022-04-02 15:22:36 (57.1 MB/s) - ‘data/catatan_lokasi.csv.2’ saved [73132/73132]

--2022-04-02 15:22:36--  https://raw.githubusercontent.com/dewirizki/DSU/main/Dataset/data_profil.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP req

In [ ]:
df1 = pd.read_csv("/content/data/catatan_lokasi.csv")
df2 = pd.read_csv("/content/data/data_profil.csv")

In [ ]:
df1.head(5)

,id,tanggal,lokasi_dominan
0,1,21/05/19,Jakarta Pusat
1,2,21/05/19,Kabupaten Bogor
2,3,21/05/19,Jakarta Pusat
3,4,21/05/19,Kota Tangerang
4,5,21/05/19,Kota Bogor


In [ ]:
df1.dtypes

id                 int64
tanggal           object
lokasi_dominan    object
dtype: object

In [ ]:
df1["tanggal"]=df1["tanggal"].astype("datetime64")

In [ ]:
df1.dtypes

id                         int64
tanggal           datetime64[ns]
lokasi_dominan            object
dtype: object

In [ ]:
df1.sample(10)

,id,tanggal,lokasi_dominan
1803,4,2019-08-06,Kota Bandar Lampung
657,58,2019-05-27,Kota Bekasi
1985,86,2019-09-06,Jakarta Barat
1215,16,2019-02-06,Kota Tangerang
1009,10,2019-05-31,Kota Yogyakarta
1338,39,2019-03-06,Kota Semarang
2173,74,2019-11-06,Kota Tangerang Selatan
1666,67,2019-06-06,Kota Banda Aceh
397,98,2019-05-24,Jakarta Timur
2567,68,2019-06-15,Jakarta Utara


In [ ]:
df2.head(5)

,id,jenis_kelamin,divisi,umur
0,1,Laki-laki,Customer Service,34
1,2,Laki-laki,Data Engineer,26
2,3,Laki-laki,Software Engineer,28
3,4,Perempuan,Business Intelligence,21
4,5,Perempuan,Software Engineer,21


In [ ]:
df2["jenis_kelamin"]=df2["jenis_kelamin"].astype("category")

In [ ]:
data = df2.merge(df1, on="id", how="left")
data.head()

,id,jenis_kelamin,divisi,umur,tanggal,lokasi_dominan
0,1,Laki-laki,Customer Service,34,2019-05-21,Jakarta Pusat
1,1,Laki-laki,Customer Service,34,2019-05-22,Jakarta Pusat
2,1,Laki-laki,Customer Service,34,2019-05-23,Jakarta Pusat
3,1,Laki-laki,Customer Service,34,2019-05-24,Kota Bekasi
4,1,Laki-laki,Customer Service,34,2019-05-25,Kota Bekasi


## 1. Berapakah umur termuda dari karyawan yang tergabung dalam divisi Data Science?

In [ ]:
young = df2.groupby("divisi")["umur"].min()\
        .reset_index(name='Young Age') \
        .sort_values(['Young Age'], \
        ascending=False)
young

,divisi,Young Age
4,Marketing,24
2,Data Engineer,23
1,Customer Service,22
0,Business Intelligence,21
3,Data Science,21
5,Software Engineer,21


In [ ]:
fig1 = px.bar(young, x='divisi', y="Young Age", title = 'Young Age per Division',width=900, height=700, color="divisi")
fig1.show()

In [ ]:
karyawan_DS_muda = young.loc[young.divisi == "Data Science"]
karyawan_DS_muda

,divisi,Young Age
3,Data Science,21


## 2. Divisi apa yang karyawannya paling banyak melakukan mudik? 

In [ ]:
jabodetabek = ['Jakarta Pusat', 'Kabupaten Bogor', 'Kota Tangerang', 'Kota Bogor',
       'Kota Depok', 'Jakarta Timur', 'Kota Bekasi', 'Jakarta Selatan',
       'Jakarta Utara', 'Jakarta Barat', 'Kabupaten Tangerang',
       'Kabupaten Bekasi', 'Kota Tangerang Selatan']

In [ ]:
mudik = data.loc[~data.lokasi_dominan.isin(jabodetabek)]

In [ ]:
most_mudik = mudik.groupby("divisi")["id"].unique().agg([len])
most_mudik

,len
divisi,
Business Intelligence,20
Customer Service,15
Data Engineer,13
Data Science,10
Marketing,8
Software Engineer,27


In [ ]:
fig3 = px.bar(most_mudik, x=most_mudik.index, y="len", title = 'Divisi yang melakukan Mudik', width=900, height=700, color="len")
fig3.show()

##  3. Berapakah median umur dari keseluruhan karyawan Tokolapak? 

In [ ]:
median_umur = df2.umur.median()
print(f'Median umur seluruh karyawan Tokolapak adalah {median_umur}')

Median umur seluruh karyawan Tokolapak adalah 27.0


## 4. Tanggal berapakah menjadi hari favorit karyawan Tokolapak pulang ke Jabodetabek?

In [ ]:
go_jabodetabek = data.loc[data.lokasi_dominan.isin(jabodetabek)]

In [ ]:
pulski = go_jabodetabek.groupby("tanggal").id.unique().agg([len])
pulski

,len
tanggal,
2019-01-06,27
2019-02-06,17
2019-03-06,7
2019-04-06,9
2019-05-06,9
2019-05-21,100
2019-05-22,100
2019-05-23,97
2019-05-24,98


In [ ]:
fig2 = px.scatter(pulski, x=pulski.index, y="len", title = 'Favorite Tanggal', width=900, height=700, color="len")
fig2.show()

##  5. Berapa jumlah karyawan yang mudik dan tergabung dalam Divisi Data Science?

In [ ]:
karyawan_DS_mudik = mudik.loc[mudik.divisi == "Data Science"].id.unique().size
print(f'Jumlah karyawan data science yang mudik sebanyak {karyawan_DS_mudik} orang')

Jumlah karyawan data science yang mudik sebanyak 10 orang


## 6. Berapakah umur karyawan yang paling senior dari karyawan yang tergabung dalam divisi  Data Engineer? 

In [ ]:
old = df2.groupby("divisi")["umur"].max()\
        .reset_index(name='Old Age') \
        .sort_values(['Old Age'], \
        ascending=False)
old

,divisi,Old Age
0,Business Intelligence,35
1,Customer Service,35
2,Data Engineer,35
3,Data Science,35
4,Marketing,35
5,Software Engineer,34


In [ ]:
karyawan_DE_old = old.loc[old.divisi == "Data Engineer"]
karyawan_DE_old

,divisi,Old Age
2,Data Engineer,35


## 7. Berapa jumlah karyawan TokoLapak? 

In [ ]:
jumlah_karyawan = data.id.unique().size
print(f'Jumlah Karyawan Tokolapak adalah {jumlah_karyawan} orang')

Jumlah Karyawan Tokolapak adalah 100 orang


## 8. Kabupaten/Kota manakah yang menjadi tujuan mudik terbanyak? 

In [ ]:
fav_lokasi = data.groupby("lokasi_dominan").id.unique().agg([len])\
            .reset_index() \
            .sort_values(['len'], \
                         ascending=False)
fav_lokasi            

,lokasi_dominan,len
19,Kota Depok,32
25,Kota Tangerang,29
2,Jakarta Selatan,26
18,Kota Bogor,26
3,Jakarta Timur,25
4,Jakarta Utara,24
0,Jakarta Barat,23
17,Kota Bekasi,23
26,Kota Tangerang Selatan,22
1,Jakarta Pusat,21


In [ ]:
fig4 = px.bar(fav_lokasi, x='lokasi_dominan', y="len", title = 'Favorite Lokasi Mudik', width=900, height=700, color="lokasi_dominan")
fig4.show()

## 9. Divisi apakah yang semua anggotanya mudik? 

In [ ]:
anggota_divisi = df2.groupby("divisi").id.unique().agg([len])

anggota_divisi

,len
divisi,
Business Intelligence,21
Customer Service,15
Data Engineer,14
Data Science,10
Marketing,9
Software Engineer,31


In [ ]:
most_mudik

,len
divisi,
Business Intelligence,20
Customer Service,15
Data Engineer,13
Data Science,10
Marketing,8
Software Engineer,27


In [ ]:
anggota_mudik = anggota_divisi.subtract(most_mudik)
anggota_mudik.loc[anggota_mudik.len == 0]

,len
divisi,
Customer Service,0
Data Science,0


In [ ]:
fig4 = px.bar(anggota_mudik, x=anggota_mudik.index, y="len", title = 'Anggota Divisi Mudik', width=900, height=700)
fig4.show()

## 10. Kabupaten/Kota manakah yang menjadi tujuan mudik paling sedikit?

In [ ]:
fav_lokasi = data.groupby("lokasi_dominan").id.unique().agg([len])\
            .reset_index() \
            .sort_values(['len'], \
                         ascending=False)
notfav_lokasi = fav_lokasi.groupby("lokasi_dominan").len.agg([min])\
            .reset_index() \
            .sort_values(['min'], \
                         ascending=True)
notfav_lokasi

,lokasi_dominan,min
8,Kabupaten Kebumen,2
14,Kota Banda Aceh,2
12,Kabupaten Tasikmalaya,4
23,Kota Semarang,5
20,Kota Manado,5
24,Kota Surabaya,6
15,Kota Bandar Lampung,7
13,Kabupaten Wonogiri,8
22,Kota Padang,8
16,Kota Bandung,8


## 11. Di hari apakah yang menjadi hari favorit (paling banyak) karyawan Tokolapak melakukan  mudik? Asumsikan waktu yang dibutuhkan untuk mudik kurang dari 1 hari, dan apabila dia  mulai berada di luar Jabodetabek pada hari tersebut, maka hari tersebutlah yang menjadi  dominan lokasi seseorang mudik. 

In [ ]:
mudik.lokasi_dominan = "Luar JABODETABEK"
go_jabodetabek.lokasi_dominan = "JABODETABEK"

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fav = pd.concat([mudik, go_jabodetabek], axis=0)
fav.head(5)

,id,jenis_kelamin,divisi,umur,tanggal,lokasi_dominan
5,1,Laki-laki,Customer Service,34,2019-05-26,Luar JABODETABEK
7,1,Laki-laki,Customer Service,34,2019-05-28,Luar JABODETABEK
8,1,Laki-laki,Customer Service,34,2019-05-29,Luar JABODETABEK
9,1,Laki-laki,Customer Service,34,2019-05-30,Luar JABODETABEK
10,1,Laki-laki,Customer Service,34,2019-05-31,Luar JABODETABEK


In [ ]:
print(fav.lokasi_dominan.unique())
print(data.shape)

['Luar JABODETABEK' 'JABODETABEK']
(2600, 6)


In [ ]:
fav_date = fav.groupby(["tanggal","lokasi_dominan"]).id.agg([len])\
          .sort_values(['tanggal'], \
          ascending=True)
fav_date

len
tanggal    lokasi_dominan       
2019-01-06 JABODETABEK        27
           Luar JABODETABEK   73
2019-02-06 JABODETABEK        17
           Luar JABODETABEK   83
2019-03-06 JABODETABEK         7
           Luar JABODETABEK   93
2019-04-06 JABODETABEK         9
           Luar JABODETABEK   91
2019-05-06 JABODETABEK         9
           Luar JABODETABEK   91
2019-05-21 JABODETABEK       100
2019-05-22 JABODETABEK       100
2019-05-23 JABODETABEK        97
           Luar JABODETABEK    3
2019-05-24 JABODETABEK        98
           Luar JABODETABEK    2
2019-05-25 JABODETABEK        98
           Luar JABODETABEK    2
2019-05-26 JABODETABEK        95
           Luar JABODETABEK    5
2019-05-27 JABODETABEK        95
           Luar JABODETABEK    5
2019-05-28 Luar JABODETABEK    9
           JABODETABEK        91
2019-05-29 JABODETABEK        81
           Luar JABODETABEK   19
2019-05-30 JABODETABEK        66
           Luar JABODETABEK   34
2019-05-31 JABODETABEK        42
           Luar JABODETABEK   58
2019-06-06 JABODETABEK         9
           Luar JABODETABEK   91
2019-06-13 JABODETABEK        97
           Luar JABODETABEK    3
2019-06-14 JABODETABEK       100
2019-06-15 JABODETABEK       100
2019-07-06 JABODETABEK         9
           Luar JABODETABEK   91
2019-08-06 JABODETABEK        17
           Luar JABODETABEK   83
2019-09-06 JABODETABEK        66
           Luar JABODETABEK   34
2019-10-06 JABODETABEK        81
           Luar JABODETABEK   19
2019-11-06 JABODETABEK        98
           Luar JABODETABEK    2
2019-12-06 JABODETABEK        98
           Luar JABODETABEK    2

## 12. Divisi apakah yang persentase pemudiknya paling sedikit ? 

In [ ]:
anggota_mudik.groupby("divisi").len.agg([max])\
          .sort_values(['max'], \
          ascending=False)

,max
divisi,
Software Engineer,4
Business Intelligence,1
Data Engineer,1
Marketing,1
Customer Service,0
Data Science,0


## 13. Divisi apakah yang jumlah pemudiknya paling sedikit ?

In [ ]:
most_mudik.groupby("divisi").len.agg([min])\
          .sort_values(['min'], \
          ascending=True)

,min
divisi,
Marketing,8
Data Science,10
Data Engineer,13
Customer Service,15
Business Intelligence,20
Software Engineer,27
